<a href="https://colab.research.google.com/github/arrojuh/python_learning/blob/main/orig_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pandas as pd
import time

In [ ]:
def filtered_by_snap_dt(X, snap_date, input):
    filtered_by_snap_dt =input.where(col(x) <= snap_date).select(sparkmax(col(x))).head()[0]
    return filtered_by_snap_dt
class HydraDriver():
    def __init__(self):
        pass

In [ ]:
def filtered_by_snap_dt(X, snap_date, input):
    filtered_by_snap_dt = input.where(
	  col(x) == input.where(col(x) <= snap_date).select(sparkmax(col(x)))).head()[0]
	  )
	  return filtered_by_snap_dt

class EnrichWCISSSponsorCptDriver(HydraDriver):
    def appending_columns_manual_cpt(self, source_df: DataFrame) -> DataFrame:
        rpt_dt = self.dates.short_date(self.dates.report_date)

        appending_columns_manual_cpt_df = source_df.withColumn(
            "rpt_dt", f.lit(rpt_dt)
        ).selectExpr(
            "*",
            "'Manual' as sor_id",
            "CASE WHEN isnull(src_counterparty_type) THEN 'CORP' ELSE src_counterparty_type END AS counterparty_type",
            "CASE WHEN isnull(src_cp_delinquency_status) THEN 'S' ELSE src_cp_delinquency_status END AS cp_delinquency_status",
            "CASE WHEN isnull(src_scel_cp_type) THEN 'company' ELSE src_sccl_cp_type END AS sccl_cp_type",
            "CASE WHEN isnull(src_sccl_cp_class)  THEN '0' ELSE src_sccl_cp_class END AS sccl_cp_class",
            "CASE WHEN isnull(src_sccl_cp_exemp_status) THEN '0' ELSE src_sccl_cp_exemp_status END AS sccl_cp_exemp_status",
            "CASE WHEN isnull(src_country_code) THEN 'USA' ELSE src_country_code END AS country_code",
            "date_format(to_timestamp(current_timestamp(), 'yyyyMMddHHmmss'), 'yyMMddHHmmss') AS publn_id"
        )

        return appending_columns_manual_cpt_df


def select_manual_cpt(self, manual_cpt_df: DataFrame) -> DataFrame:
    manual_cpt_df_filtered = manual_cpt_df.select(
        col("rpt_dt"),
        col("sor_id"),
        col("counterparty_id"),
        col("counterparty_name"),
        col("counterparty_type"),
        col("parent_id"),
        col("parent_name"),
        col("ultimate_parent_id"),
        col("ultimate_parent_name"),
        col("cp_delinquency_status"),
        col("sccl_cp_type"),
        col("sccl_cp_class"),
        col("sccl_cp_exemp_status"),
        col("total_assets"),
        col("country_code"),
        col("obligor_rating"),
        col("postal_code"),
        col("naics_type_cd"),
        col("oecd_entry_ind"),
        col("soverign_dflt_ind"),
        col("risk_weight"),
        col("publn_id")
    )

    return manual_cpt_df_filtered


def select_unique_cpt_id(self, manual_cpt_df_filtered: DataFrame) -> DataFrame:
    manual_ct_df_dedup = self.dedup(manual_cpt_df_filtered, ["counterparty_id"])
    return manual_ct_df_dedup

def select_wcis_customer(self, wcis_customer_df: DataFrame) -> DataFrame:
    wcis_customer_df_filtered = wcis_customer_df.select(
        col("wcis_cust_id"),
        col("wcis_cnsmr_nm"),
        col("entry"),
        col("oblgr_ratg_cd"),
        col("pstl_cd")
    )
    return wcis_customer_df_filtered

def filtered_by_snap_dt_block_customer_sponsor(self, block_customer_sponsor: DataFrame) -> DataFrame:
    snap_date = self.dates.report_date.isoformat()

    block_customer_sponsor_filtered = block_customer_sponsor.select(
        col("wcis_cust_id"),
        col("sponsorid"),
        col("sponsor_name"),
        col("cust_sor_id"),
        col("snap_dt")
    )

    snap_date = self.dates.report_date.isoformat()
    block_customer_sponsor_filtered_snap_dt = filtered_by_snap_dt(
    "snap_dt", snap_date, block_customer_sponsor_filtered
    )

    return block_customer_sponsor_filtered_snap_dt


def filtered_by_snap_dt_wcis_customer_naics_type(self, wcis_customer_sponsor: DataFrame) -> DataFrame:
    wcis_customer_sponsor_filtered = wcis_customer_sponsor.select(
        col("wcis_cust_id"),
        col("naics_type_cd"),
        col("snap_dt")
    )

    snap_date = self.dates.report_date.isoformat()
    wcis_customer_sponsor_filtered_snap_dt = self.filtered_by_snap_dt(
        "snap_dt", snap_date, wcis_customer_sponsor_filtered
    )

    return wcis_customer_sponsor_filtered_snap_dt

def filter_acbs_account(self, acbs_account: DataFrame) -> DataFrame:
    filter_acbs = acbs_account.filter(
        (upper(trim(col("open_ind"))) == 'Y') &
        (upper(trim(col("cust_acct_role"))) == 'PR') &
        (upper(trim(col("dept_lvl_3b_desc"))) == 'MUNICIPAL')
    )

    acbs_account_filtered = filter_acbs.select(
        col("wcis_cust_id").alias("wcis_cust_id_act"),
        col("acct_num"),
        col("dept_lvl_3b_desc"),
        col("document_description")
    )

    return acbs_account_filtered

def filtered_by_snap_dt_block_wcis_address(self, block_wcis_address_src: DataFrame) -> DataFrame:
    block_wcis_address_src_filtered = block_wcis_address_src.select(
        col("wcis_cust_id"),
        col("State"),
        col("snap_dt")
    )

    snap_date = self.dates.report_date.isoformat()
    block_wcis_address_src_filtered_snap_dt = self.filtered_by_snap_dt(
        "snap_dt", snap_date, block_wcis_address_src_filtered
    )

    return block_wcis_address_src_filtered_snap_dt

    def filtered_by_snap_dt_wcis_customer_naics_type(
        self, wcis_customer_sponsor
    ) -> DataFrame:
        wcis_customer_sponsor_filtered = wcis_customer_sponsor.select(
            col("wcis_cust_id"), col("naics_type_cd"), col("snap_dt")
        )

        snap_date = self.dates.report_date.isoformat()
        wcis_customer_sponsor_filtered_snap_dt = filtered_by_snap_dt(
            "snap_dt", snap_date, wcis_customer_sponsor_filtered
        )

        return wcis_customer_sponsor_filtered_snap_dt

    def filter_acbs_account(self, acbs_account) -> DataFrame:
        filter_acbs = acbs_account.filter(
            (upper(trim(acbs_account.open_ind)) == 'Y')
            & (upper(trim(acbs_account.cust_acct_role)) == 'PR')
            & (upper(trim(acbs_account.dept_lvl_3b_desc)) == 'MUNICIPAL')
        )

        acbs_account_filtered = filter_acbs.select(
            col("wcis_cust_id").alias("wcis_cust_id_act"),
            col("acct_num"),
            col("dept_lvl_3b_desc"),
            col("document_description"),
        )

        return acbs_account_filtered

    def filtered_by_snap_dt_block_wcis_address(
        self, block_wcis_address_src
    ) -> DataFrame:
        block_wcis_address_src_filtered = block_wcis_address_src.select(
            col("wcis_cust_id"), col("State"), col("snap_dt")
        )

        snap_date = self.dates.report_date.isoformat()
        block_wcis_address_src_filtered_snap_dt = filtered_by_snap_dt(
            "snap_dt", snap_date, block_wcis_address_src_filtered
        )

        return block_wcis_address_src_filtered_snap_dt

    def select_unique_wcis_cust_id(self, wcis_customer) -> DataFrame:
        wcis_customer_df_dedup = self.dedup(wcis_customer, ["wcis_cust_id"])

        return wcis_customer_df_dedup

    def select_unique_wcis_customer_naics_type(
        self, wcis_customer_naics_type
    ) -> DataFrame:
        wcis_customer_naics_type_df_dedup = self.dedup(
            wcis_customer_naics_type, ["wcis_cust_id", "naics_type_cd"]
        )

        return wcis_customer_naics_type_df_dedup

    def select_unique_block_wcis_address(self, block_wcis_address) -> DataFrame:
        block_wcis_address_df_dedup = self.dedup(block_wcis_address, ["wcis_cust_id"])

        return block_wcis_address_df_dedup

    def multi_join_wcis_minerva_relationship(
        self, wcis_minerva_relationship, joinedds3
    ) -> DataFrame:
        join_table = wcis_minerva_relationship.join(
            joinedds3,
            wcis_minerva_relationship.PARENT_MINERVA_ID == joinedds3.instun_id,
            "left_outer"
        )

        wcis_minerva_relationship_with_name = join_table.select(
            "wcis_id",
            col("PARENT_MINERVA_ID").alias("parent_minerva_id"),
            "instun_id",
            "sor_instun_id",
            "instun_nm",
            "cntry_cd",
            "instn_typ",
            "cnterpty_typ1",
            "cnterpty_typ2",
            "instun_ultimate_parent_nm",
            "instun_ultimate_parent_id",
            "instun_dirct_parnt_id",
            "instun_type_id",
            "instun_dirct_parnt_nm",
            "cnterpty_cls_id",
            "gts_country",
            "country_risk_category",
            "cnterpty_exempt_stat_cd",
        )

        return wcis_minerva_relationship_with_name

        def multi_join_acbs_account_with_docnstate(
        self, acbs_account, block_wcis_address
    ) -> DataFrame:
        join_table = acbs_account.join(
            block_wcis_address,
            acbs_account.wcis_cust_id_act == block_wcis_address.wcis_cust_id,
            "left_outer"
        )

        acbs_account_with_docnstate = join_table.select(
            "wcis_cust_id_act",
            "acct_num",
            "dept_lvl_3b_desc",
            "State",
            "document_description",
            "wcis_cust_id"
        )

        return acbs_account_with_docnstate

    def multi_join_acbs_account_with_docnstate_psecpt(
        self, acbs_account_with_docnstate, manual_counterparty
    ) -> DataFrame:
        join_table = acbs_account_with_docnstate.join(
            manual_counterparty,
            acbs_account_with_docnstate.State == manual_counterparty.counterparty_id,
            "left_outer"
        )

        acbs_account_with_docnstate_psecpt = join_table.select(
            "wcis_cust_id_act",
            "acct_num",
            "dept_lvl_3b_desc",
            "State",
            col("counterparty_id").alias("counterparty_id_pse"),
            col("counterparty_name").alias("counterparty_name_pse"),
            col("counterparty_type").alias("counterparty_type_pse"),
            col("parent_id").alias("parent_id_pse"),
            col("parent_name").alias("parent_name_pse"),
            col("ultimate_parent_id").alias("ultimate_parent_id_pse"),
            col("ultimate_parent_name").alias("ultimate_parent_name_pse"),
            col("sccl_cp_type").alias("sccl_cp_type_pse"),
            "document_description"
        )

        return acbs_account_with_docnstate_psecpt

        def multi_join_3dspse(
        self, wcis_customer, acbs_account_with_docnstate_psecpt, cra
    ) -> DataFrame:
        join_table = wcis_customer.join(
            acbs_account_with_docnstate_psecpt,
            wcis_customer.wcis_cust_id
            == acbs_account_with_docnstate_psecpt.wcis_cust_id_act,
            "left_outer"
        ).join(
            cra,
            wcis_customer.wcis_cust_id == cra.cra_cnterpty_cd,
            "left_outer",
        )

        joineddspse = join_table.select(
            col("wcis_cust_id").alias("wcis_cust_id_cust"),
            "wcis_cust_id_act",
            "wcis_cnsmr_nm",
            "cntry",
            "oblgr_ratg_cd",
            "pstl_cd",
            "acct_num",
            "dept_lvl_3b_desc",
            "State",
            "counterparty_id_pse",
            "counterparty_name_pse",
            "counterparty_type_pse",
            "parent_id_pse",
            "parent_name_pse",
            "ultimate_parent_id_pse",
            "ultimate_parent_name_pse",
            "sccl_cp_type_pse",
            "cra_expsr_id",
            "cra_cnterpty_cd",
            "cra_imp_src_id",
            "cra_PRIM_DEVLPR_NM",
            "document_description"
        )

        return joineddspse


        def multi_join_joined_dspseminerva(
        self, joineddspse, wcis_minerva_relationship_with_name
    ) -> DataFrame:
        join_table = joineddspse.join(
            wcis_minerva_relationship_with_name,
            joineddspse.wcis_cust_id_cust
            == wcis_minerva_relationship_with_name.wcis_id,
            "left_outer"
        )

        joineddspse_minerva = join_table.select(
            "wcis_cust_id_cust",
            "wcis_id",
            "wcis_cnsmr_nm",
            "cntry",
            "oblgr_ratg_cd",
            "pstl_cd",
            "acct_num",
            "dept_lvl_3b_desc",
            "State",
            "counterparty_id_pse",
            "counterparty_name_pse",
            "counterparty_type_pse",
            "parent_id_pse",
            "parent_name_pse",
            "ultimate_parent_id_pse",
            "ultimate_parent_name_pse",
            "sccl_cp_type_pse",
            "parent_minerva_id",
            "instun_nm",
            "sor_instun_id",
            "instn_typ",
            "cnterpty_typ1",
            "cnterpty_typ2",
            "instun_ultimate_parent_nm",
            "instun_ultimate_parent_id",
            "instun_dirct_parnt_id",
            "instun_type_id",
            "instun_dirct_parnt_nm",
            "cnterpty_cls_id",
            "gts_country",
            "cnterpty_exmpt_stat_cd",
            "country_risk_category",
            "cra_expsr_id",
            "cra_cnterpty_cd",
            "cra_imp_src_id",
            "cra_PRIM_DEVLPR_NM",
            "document_description",
        )

        return joineddspse_minerva

    def multi_join_pse_ds(
        self, joineddspse_minerva, block_customer_sponsor
    ) -> DataFrame:
        join_table = joineddspse_minerva.join(
            block_customer_sponsor,
            joineddspse_minerva.wcis_cust_id_cust
            == block_customer_sponsor.wcis_cust_id,
            "left_outer"
        )

        pse_ds = join_table.select(
            "wcis_cust_id_cust",
            "wcis_cnsmr_nm",
            "wcis_cust_id",
            "acct_num",
            "dept_lvl_3b_desc",
            "State",
            "counterparty_id_pse",
            "counterparty_name_pse",
            "counterparty_type_pse",
            "parent_id_pse",
            "parent_name_pse",
            "ultimate_parent_id_pse",
            "ultimate_parent_name_pse",
            "sccl_cp_type_pse",
            "parent_minerva_id",
            "instun_nm",
            "cntry",
            "oblgr_ratg_cd",
            "pstl_cd",
            "sponsorid",
            "sponsor_name",
            "cust_sor_id",
            "snap_dt",
            "cnterpty_typ1",
            "cnterpty_typ2",
            "cnterpty_cls_id",
            "gts_country",
            "cnterpty_exmpt_stat_cd",
            "country_risk_category",
            "cra_expsr_id",
            "cra_cnterpty_cd",
            "cra_imp_src_id",
            "cra_PRIM_DEVLPR_NM",
            "document_description",
        )

        return pse_ds

    def multi_join_sponsor_ds(
        self, joineddspse_minerva, block_customer_sponsor
    ) -> DataFrame:
        join_table = joineddspse_minerva.join(
            block_customer_sponsor,
            joineddspse_minerva.wcis_cust_id_cust == block_customer_sponsor.sponsorid,
            "inner"
        )

        sponsor_ds = join_table.select(
            "wcis_cust_id_cust",
            "wcis_cnsmr_nm",
            "acct_num",
            "dept_lvl_3b_desc",
            "State",
            "counterparty_id_pse",
            "counterparty_name_pse",
            "counterparty_type_pse",
            "parent_id_pse",
            "parent_name_pse",
            "ultimate_parent_id_pse",
            "ultimate_parent_name_pse",
            "sccl_cp_type_pse",
            "parent_minerva_id",
            "instun_nm",
            "cntry",
            "oblgr_ratg_cd",
            "pstl_cd",
            "sponsorid",
            "sponsor_name",
            "cust_sor_id",
            "snap_dt",
            "cnterpty_typ1",
            "cnterpty_typ2",
            "cnterpty_cls_id",
            "gts_country",
            "cnterpty_exmpt_stat_cd",
            "country_risk_category",
            "cra_expsr_id",
            "cra_cnterpty_cd",
            "cra_imp_src_id",
            "cra_PRIM_DEVLPR_NM",
            "document_description",
            col("parent_minerva_id").alias("parent_minerva_id_sponsor"),
            col("cnterpty_cls_id").alias("cnterpty_cls_id_sponsor"),
            col("cnterpty_exmpt_stat_cd").alias("cnterpty_exmpt_stat_cd_sponsor"),
        )

        return sponsor_ds

    def join_sponsor_pse(self, pse_ds, sponsor_ds) -> DataFrame:
        sponsor_ds = sponsor_ds.select(
            col("sponsorid").alias("sponsorid_sponsor_ds"),
            "parent_minerva_id_sponsor",
            "cnterpty_cls_id_sponsor",
            "cnterpty_exmpt_stat_cd_sponsor"
        )

        join_table = pse_ds.join(
            sponsor_ds,
            pse_ds.sponsorid == sponsor_ds.sponsorid_sponsor_ds,
            "left_outer"
        )

            joined_table = join_table.select(
        "wcls_cust_id_cust",
        "wcls_cnsnr_nm",
        "acct_num",
        "dept_lvl_3b_desc",
        "State",
        "counterparty_id_pse",
        "counterparty_name_pse",
        "counterparty_type_pse",
        "parent_id_pse",
        "parent_name_pse",
        "ultimate_parent_id_pse",
        "ultimate_parent_name_pse",
        "sccl_cp_type_pse",
        "parent_minerva_id",
        "instun_nm",
        "cntry",
        "oblgr_ratg_cd",
        "pstl_cd",
        "sponsorid",
        "sponsor_name",
        "cust_sor_id",
        "snap_dt",
        "cnterpty_typ1",
        "cnterpty_typ2",
        "cnterpty_cls_id",
        "gts_country",
        "cnterpty_exmpt_stat_cd",
        "country_risk_category",
        "cra_expsr_id",
        "cra_cnterpty_cd",
        "cra_imp_src_id",
        "cra_PRIM_DEVLPR_NM",
        "document_description",
        "parent_minerva_id_sponsor",
        "cnterpty_cls_id_sponsor",
        "cnterpty_exmpt_stat_cd_sponsor",
    )

    return joined_table


    def join_sponsor_pse_final(self, pse, spon) -> DataFrame:
        spon = spon.select(
            col("instun_nm").alias("instun_nm_sponsor"),
            "sor_instun_id",
        )

        pse = pse.select(
            pse.wcls_cust_id_cust,
            pse.wcls_cnsnr_nm,
            pse.acct_num,
            pse.dept_lvl_3b_desc,
            pse.State,
            pse.counterparty_id_pse,
            pse.counterparty_name_pse,
            pse.counterparty_type_pse,
            pse.parent_id_pse,
            pse.parent_name_pse,
            pse.ultimate_parent_id_pse,
            pse.ultimate_parent_name_pse,
            pse.sccl_cp_type_pse,
            pse.parent_minerva_id,
            pse.instun_nm,
            pse.cntry,
            pse.oblgr_ratg_cd,
            pse.pstl_cd,
            pse.sponsorid,
            pse.sponsor_name,
            pse.cust_sor_id,
            pse.snap_dt,
            pse.cnterpty_typ1,
            pse.cnterpty_typ2,
            pse.cnterpty_cls_id,
            pse.gts_country,
            pse.cnterpty_exmpt_stat_cd,
            pse.country_risk_category,
            pse.cra_expsr_id,
            pse.cra_cnterpty_cd,
            pse.cra_imp_src_id,
            pse.cra_PRIM_DEVLPR_NM,
            pse.document_description,
            pse.parent_minerva_id_sponsor,
            pse.cnterpty_cls_id_sponsor,
            pse.cnterpty_exmpt_stat_cd_sponsor,
            concat(lit('196-'), pse.parent_minerva_id_sponsor).alias(
                "parent_minerva_id_sponsor2"
            ),
        )

        joined_table = pse.join(
            spon,
            pse.parent_minerva_id_sponsor2 == spon.sor_instun_id,
            "left_outer"
        ).drop(pse.parent_minerva_id_sponsor2)

        return joined_table

    def multi_join_joinedds2(
        self, sponsor_pse_final, wcis_customer_naics_type2
    ) -> DataFrame:
        wcis_customer_naics_type = wcis_customer_naics_type2.drop('snap_dt')

        join_table = sponsor_pse_final.join(
            wcis_customer_naics_type,
            sponsor_pse_final.wcis_cust_id_cust == wcis_customer_naics_type.wcis_cust_id,
            "left_outer"
        )

        joinedds2 = join_table.select(
            "wcis_cust_id_cust",
            "wcis_cust_id",
            "wcls_cnsnr_nm",
            "acct_num",
            "dept_lvl_3b_desc",
            "counterparty_id_pse",
            "counterparty_name_pse",
            "counterparty_type_pse",
            "parent_id_pse",
            "parent_name_pse",
            "ultimate_parent_id_pse",
            "ultimate_parent_name_pse",
            "sccl_cp_type_pse",
            "parent_minerva_id",
            "instun_nm",
            "instun_nm_sponsor",
            "cntry",
            "oblgr_ratg_cd",
            "pstl_cd",
            "sponsorid",
            "sponsor_name",
            "cust_sor_id",
            "snap_dt",
            "naics_type_cd",
            "cnterpty_typ1",
            "cnterpty_typ2",
            "cnterpty_tls_id",
            "gts_country",
            "cnterpty_exmpt_stat_cd",
            "country_risk_category",
            "cra_expsr_id",
            "cra_cnterpty_cd",
            "cra_imp_src_id",
            "cra_PRIN_DEVLPR_NN",
            "document_description",
            "parent_minerva_id_sponsor",
            "cnterpty_cls_id_sponsor",
            "cnterpty_exmpt_stat_cd_sponsor"
        )

        return joinedds2

    def multi_join_joinedds5(self, joinedds2, block_wcis_address_state) -> DataFrame:
        block_wcis_address_state = block_wcis_address_state.select(
            col("wcis_cust_id").alias("wcis_cust_id_join"), "State"
        )

        join_table = joinedds2.join(
            block_wcis_address_state,
            joinedds2.wcis_cust_id_cust == block_wcis_address_state.wcis_cust_id_join,
            "left_outer"
        )

        joinedds5 = join_table.select(
            col("wcis_cust_id_cust").alias("wcis_cust_id"),
            "wcis_cust_id_join",
            "wcls_cnsnr_nm",
            "acct_num",
            "dept_lvl_3b_desc",
            "State",
            "counterparty_id_pse",
            "counterparty_name_pse",
            "counterparty_type_pse",
            "parent_id_pse",
            "parent_name_pse",
            "ultimate_parent_id_pse",
            "ultimate_parent_name_pse",
            "secl_cp_type_pse",
            "parent_minerva_id",
            "instun_nm",
            "instun_nm_sponsor",
            "cntry",
            "oblgr_ratg_cd",
            "pstl_cd",
            "sponsorid",
            "sponsor_name",
            "cust_sor_id",
            "snap_dt",
            "naics_type_cd",
            "cnterpty_typ1",
            "cnterpty_typ2",
            "cnterpty_tls_id",
            "gts_country",
            "cnterpty_exmpt_stat_cd",
            "country_risk_category",
            "cra_expsr_id",
            "cra_cnterpty_cd",
            "cra_imp_src_id",
            "cra_PRIN_DEVLPR_NN",
            "document_description",
            "parent_minerva_id_sponsor",
            "cnterpty_cls_id_sponsor",
            "cnterpty_exmpt_stat_cd_sponsor",
        )

        return joinedds5

    def append_fields_ds5(self, df1) -> DataFrame:
        report_date = self.dates.report_date.strftime("%Y%m%d")

                df2 = df1.select(
            "#",
            lit(report_date).cast(StringType()).alias("rpt_dt"),
            lit("123").cast(StringType()).alias("sor_id"),
            concat(lit("123-"), col("wcis_cust_id"))
                .cast(StringType())
                .alias("counterparty_id"),
        )

        df3 = df2.select(
            "#",
            when(
                isnull(col("wcis_ensar_nm")),
                concat(lit("123-"), col("wcis_cust_id")),
            )
            .otherwise(col("wcis_ensar_nm"))
            .cast(StringType())
            .alias("cust_leg_nm"),
        )

        df4 = df3.select(
            "#",
            col("cust_leg_nm").cast(StringType()).alias("counterparty_name"),
        )

        df5 = df4.selectExpr(
            "#",
            "case WHEN isnotnull(document_description) THEN parent_id_pse WHEN parent_minerva_id is not null THEN concat('196-',parent_minerva_id) ELSE NULL end as parent_id_pse",
            "case WHEN isnotnull(document_description) THEN parent_name_pse WHEN parent_minerva_id is not null THEN instun_nm ELSE NULL end as parent_name_pse",
            "case WHEN isnotnull(document_description) THEN ultimate_parent_id_pse WHEN sponsor_name is not null AND parent_minerva_id is not null THEN concat('196-',parent_minerva_id) ELSE NULL end as ultimate_parent_id_pse",
            "case WHEN isnotnull(document_description) THEN ultimate_parent_name_pse WHEN sponsor_name is not null AND parent_minerva_id is not null THEN instun_nm_sponsor ELSE NULL end as ultimate_parent_name_pse",
            "case WHEN isnotnull(dept_lvl_3b_desc) THEN 'PSE' ELSE 'CORP' end as counterparty_type",
            "is_cp_eligibility_status",
            "case WHEN isnotnull(dept_lvl_3b_desc) THEN 'U.S. state or municipality' when upper(cnterpty_typ1) in ('BANK','SFI') THEN 'Bank' ELSE 'Corporate' end as counterparty_type1",
            "case WHEN sponsorid is NOT NULL and parent_minerva_id_sponsor is NOT NULL then nvl(cnterpty_cls_id_sponsor,cnterpty_exmpt_stat_cd_sponsor) ELSE NULL end as cnterpty_cls_id_sponsor",
        )

        final = df5.select(
            "#",
            lit(None).cast(DoubleType()).alias("total_assets"),
            lit(None).cast(StringType()).alias("oecd_cntry_ind"),
            lit(None).cast(StringType()).alias("sovereign_dflt_ind"),
            lit(None).cast(StringType()).alias("risk_weight"),
            date_format(col("current_timestamp"),"yyyyMMddHHmmss")
            .cast(StringType())
            .alias("publn_id"),
        )
        return final

           def select_joinsDS(self, joined_ds_final) -> DataFrame:
        selected_df = joined_ds_final.select(
            "rpt_dt",
            "sor_id",
            "counterparty_id",
            "counterparty_name",
            "counterparty_type",
            "parent_id",
            "parent_name",
            "ultimate_parent_id",
            "ultimate_parent_name",
            "cp_delinquency_status",
            "secl_cp_type",
            "secl_cp_class",
            "secl_cp_exemp_status",
            "total_assets",
            col("entry").alias("country_code"),
            col("oblg_rtg_cd").alias("obligor_rating"),
            col("pstl_cd").alias("postal_code"),
            col("naics_type_cd").alias("naics_type_cd"),
            "oecd_cntry_ind",
            "sovereign_dflt_ind",
            "risk_weight",
            "publn_id",
            col("state").alias("state_code"),
            "wcis_cust_id",
        )

        return selected_df

    def filtered_metric_customer(self, metric_customer_ttcpd) -> DataFrame:
        metric_customer_ttcpd_filtered = metric_customer_ttcpd.select(
            trim(col("wcis_cust_id")).alias("wcis_cust_id_ttccpd"),
            "worst_obligor_rating",
            "scorecard_name",
            "snap_dt",
        )

        snap_date = self.dates.report_date.isoformat()
        metric_customer_ttcpd_filtered_snap_dt = filtered_by_snap_dt(
            "snap_dt", snap_date, metric_customer_ttcpd_filtered
        )

        return metric_customer_ttcpd_filtered_snap_dt
    def select_unique_final_ds(self, final_filtered) -> DataFrame:
        final_dedup = self.dedup(
            final_filtered, ["counterparty_id", "ultimate_parent_id"]
        )

        return final_dedup

    def join_on_final(self, finalds, ttcpd_ds) -> DataFrame:
        join_table = finalds.join(
            ttcpd_ds,
            finalds.wcis_cust_id == ttcpd_ds.wcis_cust_id_ttccpd,
            "left_outer",
        )

        return join_table

    def select_erdm_rscr_model_risk_ratg(self, erdm_rscr_model_risk_ratg):
        rscr_model_risk = erdm_rscr_model_risk_ratg.select(
            trim(col("SCORCD_ID")).alias("SCORCD_ID"),
            trim(col("DFLT_PRBLT_RATG_CD")).alias("DFLT_PRBLT_RATG_CD"),
            trim(col("SCORCD_NM")).alias("SCORCD_NM"),
            trim(col("DFLT_PRBLT_RATG_VAL")).alias("DFLT_PRBLT_RATG_VAL"),
        )

        return rscr_model_risk

    def select_final_ds_wpd(self, final_ds_wpd):
        final_ds_wpd = final_ds_wpd.select(
            "*", col("obligor_rating").alias("obligor_rating1")
        )

        final_ds_wpd_filtered = final_ds_wpd.selectExpr(
            "rpt_dt",
            "sor_id",
            "counterparty_id",
            "counterparty_name",
            "counterparty_type",
            "parent_id",
            "parent_name",
            "ultimate_parent_id",
            "ultimate_parent_name",
            "cp_delinquency_status",
            "secl_cp_type",
            "secl_cp_class",
            "secl_cp_exemp_status",
            "total_assets",
            "country_code",
            "obligor_rating",
            "worst_obligor_rating",
            "CASE WHEN (wcis_cust_id_ttccpd IS NOT NULL or length(trim(wcis_cust_id_ttccpd)) = 0) THEN worst_obligor_rating ELSE NULL END as worst_obligor_rating",
            "postal_code",
            "naics_type_cd",
            "geol_cntry_ind",
            "sovereign_dflt_ind",
            "wkc_weight",
            "publn_id",
            "state_code",
            "scorecard_name",
            "CASE WHEN (wcis_cust_id_ttccpd IS NOT NULL or length(trim(wcis_cust_id_ttccpd)) = 0) THEN scorecard_name ELSE NULL END as scorecard_name",
            "wcis_cust_id_ttccpd",
            "wcis_cust_id",
        )

        return final_ds_wpd_filtered

    def join_on_final_erdm(self, finalds, erdm_rscr_model_risk_ratg) -> DataFrame:
        join_table = finalds.join(
            erdm_rscr_model_risk_ratg,
            (finalds.obligor_rating == erdm_rscr_model_risk_ratg.DFLT_PRBLT_RATG_CD)
            & (finalds.BONUS_FIELD_1 == erdm_rscr_model_risk_ratg.SCORCD_NM),
            "left_outer",
        )

        return join_table

    def append_bonus_fields(self, input):
        output = input.select(
            "*",
            col("DFLT_PRBLT_RATG_VAL").alias("BONUS_FIELD_2"),
        )
        return output

    # Method to create CREDIT_RATING, OBLIGOR_RATING_SCORECARD, OBLIGOR_RATING_VALUE
    def append_crd_rt_scr_card(self, df: DataFrame) -> DataFrame:
        return df.select(
            "*",
            when(col("worst_obligor_rating") == lit(5), "IG")
            .when(
                (col("worst_obligor_rating") == lit(6))
                & (col("worst_obligor_rating") == lit(14)),
                "SG",
                )
             .when(col("worst_obligor_rating") > lit(14), "SSG")
            .otherwise("SSG")
            .cast(StringType())
            .alias("CREDIT_RATING"),
            col("scorecard_name").cast(StringType()).alias("OBLIGOR_RATING_SCORECARD"),
            col("DFLT_PRBLT_RATG_VAL").cast(StringType()).alias("OBLIGOR_RATING_VALUE"),
        )

    def run_transformations(self, source_dfs: dict):
        logger.info("Starting ENRICH_WCIS_SPONSER_CPF")
        wcis_customer_file = source_dfs["WCIS_CUSTOMER_FILE"]  # step1
        block_customer_sponsor_file = source_dfs["BLOCK_CUSTOMER_SPONSOR"]  # step2
        wcis_cust_naics_type_file = source_dfs["WCIS_CUST_NAICS_TYPE_FILE"]  # step3
        acbs_account = source_dfs["ACBS_ACCOUNT"]  # step4
        block_wcis_address_file = source_dfs["BLOCK_WCIS_ADDRESS"]  # step5
        wcis_minerva_relationship = source_dfs["WCIS_MINERVA_RELATIONSHIP"]  # step6

        # step7
        deduped_wcis_minerva_relationship = self.dedup(
            df=wcis_minerva_relationship, dedup_fields=["WCIS_ID"]
        )

        # step9
        manual_cntrprty_file = source_dfs["MANUAL_CNTRPRTY_FILE"]

        manual_cntrprty_file = (
            manual_cntrprty_file.withColumn(
                "total_assets_temp", col("total_assets").cast(DoubleType())
            )
            .drop("total_assets")
            .withColumnRenamed("total_assets_temp", "total_assets")
        )

        # step10
        minerva_cntrprty_rltnsp_file = source_dfs["MINERVA_CNTRPRTY_RLTNSP_FILE"]

        # step11
        minerva_cntrprty_type_file = source_dfs["MINERVA_CNTRPRTY_TYPE_FILE"]

        # step12
        dedup_minerva_cntrprty_type_file = self.dedup(
            df=minerva_cntrprty_type_file, dedup_fields=["instn_typ", "cnterprty_type"]
        )

            # step13
    acbs_document = source_dfs["ACBS_DOCUMENT"]

    # step14
    cra = source_dfs["CRA"]

    cra = (
        cra.withColumn(
            "cra_outstd_bal_amt_temp", col("cra_outstd_bal_amt").cast(DoubleType())
        )
        .drop("cra_outstd_bal_amt")
        .withColumnRenamed("cra_outstd_bal_amt_temp", "cra_outstd_bal_amt")
    )

    # step15
    metric_customer_basel_ttcpd = source_dfs["METRIC_CUSTOMER_BASEL_TTCPD"]

    # step16
    cra_select = cra.select(
        col("cra_rpt_dt"),
        col("cra_expsr_id"),
        col("cra_cnterprty_cd"),
        col("cra_imp_src_id"),
        col("cra_PRIM_DEVLPR_NN"),
    )

    # step17
    acbs_document_filter_the_snap_date_df = self.acbs_document_filter_the_snap_date(
        input_df=acbs_document
    )

    # step18
    step18 = self.join_acbs_account_with_acbs_document(
        acbs_account=acbs_account,
        acbs_document=acbs_document_filter_the_snap_date_df,
    )

    # step19
    step19 = self.join_minerva_counterparty_rltnsp_with_minerva_cnterprty_typ(
        minerva_counterparty_rltnsp=minerva_cntrprty_rltnsp_file,
        minerva_cnterprty_typ=dedup_minerva_cntrprty_type_file,
    )
     # step20
    ultimate_cpty_ds = self.select_ultimate_counterparty(joined_ds_typ=step19)

    # step21
    joined_df_ult_par = self.ultimate_cpty_ds(
        step19=step19,
        ultimate_cpty_ds=ultimate_cpty_ds,
    )

    # step22
    joined_df_3 = self.select_minerva_counterparty(
        joined_ds2_ult_par=joined_df_ult_par,
        minerva_cnterprty_typ=dedup_minerva_cnterprty_type_file,
    )

    # step23
    joined_ds3_1 = self.dedup(
        df=joined_df_3, dedup_fields=["instun_id", "instun_nm"]
    )

    # step24
    minerva_cnterprty_exempt_file = source_dfs["MINERVA_CNTERPRTY_EXEMPT_FILE"]

    # step25
    dedup_minerva_cnterprty_exempt_file = self.dedup(
        df=minerva_cnterprty_exempt_file,
        dedup_fields=["instun_id", "cnterprty_cls_id", "cnterprty_exempt_stat_cd"],
    )

    # step26
    erdm_risk_clasfn_cntry_file = source_dfs["ERDM_RISK_CLASFN_CNTRY_FILE"]

    erdm_risk_clasfn_cntry_file = (
        erdm_risk_clasfn_cntry_file.withColumn(
            "risk_weight_temp", col("risk_weight").cast(IntegerType())
        )
        .drop("risk_weight")
        .withColumnRenamed("risk_weight_temp", "risk_weight")
    )

    # step27
    dedup_erdm_risk_clasfn_cntry_file = self.dedup(
        df=erdm_risk_clasfn_cntry_file,
        dedup_fields=["gts_country"],
    )
    # step28
    joined_ds3_2 = self.join_minerva_cnterprty_exempt_with_erdm(
        joined_ds3_1=joined_ds3_1,
        dedup_minerva_cnterprty_exempt_file=dedup_minerva_cnterprty_exempt_file,
        dedup_erdm_risk_clasfn_cntry_file=dedup_erdm_risk_clasfn_cntry_file,
    )

    # step29
    append_manual_counterparty = self.appending_columns_manual_cpt(
        manual_cntrprty_file
    )

    # step30
    manual_counterparty = self.select_manual_cpt(append_manual_counterparty)

    # step31
    unique_manual_counterparty = self.select_unique_cpt_id(manual_counterparty)

    # step32
    wcis_customer = self.select_wcis_customer(wcis_customer_file)

    # step33
    block_customer_sponsor = self.filtered_by_snap_dt_block_customer_sponsor(
        block_customer_sponsor_file
    )

    # step34
    wcis_customer_naics_type = self.filtered_by_snap_dt_wcis_customer_naics_type(
        wcis_cust_naics_type_file
    )

    # step35
    filter_acbs_account = self.filter_acbs_account(step18)

    # step36
    block_wcis_address = self.filtered_by_snap_dt_block_wcis_address(
        block_wcis_address_file
    )

    # step38
    wcis_customer_dedup = self.select_unique_wcis_cust_id(wcis_customer)

    # step39
    wcis_customer_naics_type_dedup = self.select_unique_wcis_customer_naics_type(
        wcis_customer_naics_type
    )
    # step40
    block_wcis_address_dedup = self.select_unique_block_wcis_address(
        block_wcis_address
    )

    # step41
    wcis_minerva_relationship_with_name = self.multi_join_wcis_minerva_relationship(
        deduped_wcis_minerva_relationship, joined_ds3_2
    )

    # step42
    acbs_account_with_docnstate = self.multi_join_acbs_account_with_docnstate(
        filter_acbs_account, block_wcis_address_dedup
    )

    # step43
    acbs_account_with_docnstate_psecpt = (
        self.multi_join_acbs_account_with_docnstate_psecpt(
            acbs_account_with_docnstate, unique_manual_counterparty
        )
    )

    # step44
    joined_ds_pse = self.multi_join_3dspse(
        wcis_customer_dedup, acbs_account_with_docnstate_psecpt, cra_select
    )

    # step45
    joined_ds_pse_minerva = self.multi_join_joined_dspseminerva(
        joined_ds_pse, wcis_minerva_relationship_with_name
    )

    # step46
    pse_ds = self.multi_join_pse_ds(joined_ds_pse_minerva, block_customer_sponsor)

    # step47
    sponsor_ds = self.multi_join_sponsor_ds(
        joined_ds_pse_minerva, block_customer_sponsor
    )

    # step48
    sponsor_pse = self.join_sponsor_pse(pse_ds, sponsor_ds)
        # step49
    sponsor_pse_final = self.join_sponsor_pse_final(sponsor_pse, joined_ds3_2)

    # step50
    joined_ds2 = self.multi_join_joinedds2(
        sponsor_pse_final, wcis_customer_naics_type_dedup
    )

    # step51
    joined_ds5 = self.multi_join_joinedds5(joined_ds2, block_wcis_address)

    # step52
    joined_out = self.append_fields_ds5(joined_ds5)

    # step53
    final_ds = self.select_joinds5(joined_out)

    # step54
    ttcpd_ds = self.filtered_metric_customer(metric_customer_basel_ttcpd)

    # step55
    unique_final_ds = self.select_unique_final_ds(final_ds)

    # step56
    final_ds_wpd = self.join_on_final(unique_final_ds, ttcpd_ds)
    self.debug("enrich_wcis_sponsor_cpt_step56_join_on_final", final_ds_wpd)

    # step58
    erdm_rscr_model_risk_ratg = self.select_erdm_rscr_model_risk_ratg(
        source_ds["ERDM_RSCR_MODEL_RISK_RATG_FILE"]
    )
    self.debug(
        "enrich_wcis_sponsor_cpt_step58_select_erdm_rscr_model_risk_ratg",
        erdm_rscr_model_risk_ratg,
    )

    # step59
    final_ds_wpd_final = self.select_final_ds_wpd(final_ds_wpd)
    self.debug(
        "enrich_wcis_sponsor_cpt_step59_select_final_ds_wpd", final_ds_wpd_final
    )

    # step60
    final_ds_wpd_final_output = self.join_on_final_erdm(
        final_ds_wpd_final, erdm_rscr_model_risk_ratg
    )

    self.debug(
        "enrich_wcis_sponsor_cpt_step60_join_on_final_erdm",
        final_ds_wpd_final_output,
    )

    # step 61 To create CREDIT_RATING, OBLIGOR_RATING_SCORECARD, OBLIGOR_RATING_VALUE & 2052a_CP_TYPE
    append_crd_rt_scr_card_df = self.append_crd_rt_scr_card(
        final_ds_wpd_final_output
    )

    self.debug(
        "enrich_wcis_sponsor_cpt_step61_append_crd_rt_scr_card",
        append_crd_rt_scr_card_df,
    )

    # step62
    final = self.append_bonus_fields(append_crd_rt_scr_card_df)
    self.debug("enrich_wcis_sponsor_cpt_step61_append_bonus_fields", final)

    # step63
    final_formatted = common.complete_dataframe(
        source_df=final,
        schema=final_schema,
    )

    self.debug("enrich_wcis_sponsor_cpt_step62_complete_dataframe", final_formatted)
    logger.info("Completed ENRICH_WCIS_SPONSER_CPT")

    return {"ENRICH_WCIS_SPONSOR_CPT": final_formatted}

    def join_minerva_cnterpty_exmpt_with_erdm(
        self,
        joined_ds3_1,
        dedup_minerva_cnterpty_exempt_file,
        dedup_erdm_risk_clasfn_cntry_file,
    ) -> DataFrame:
        join_table_temp = joined_ds3_1.join(
            dedup_minerva_cnterpty_exempt_file,
            joined_ds3_1.sor_instun_id == dedup_minerva_cnterpty_exempt_file.instn_id,
            "left_outer",
        )

        join_table = join_table_temp.join(
            dedup_erdm_risk_clasfn_cntry_file,
            join_table_temp.cntry_cd == dedup_erdm_risk_clasfn_cntry_file.risk_country,
            "left_outer",
        )
    return join_table

        def dedup(self, df: DataFrame, dedup_fields: list) -> DataFrame:
        logger.debug("Input Dataframe Count:%s", df.count())
        dropped_df = df.dropDuplicates(dedup_fields)
        logger.debug("Output Dataframe Count:%s", dropped_df.count())
        return dropped_df

    def join_minerva_counterparty_rtnsp_with_minerva_cnterpty_typ(
        self, minerva_counterparty_rtnsp, minerva_cnterpty_typ
    ) -> DataFrame:
        join_table = minerva_counterparty_rtnsp.join(
            minerva_cnterpty_typ,
            minerva_counterparty_rtnsp.instun_type_id
            == minerva_cnterpty_typ.instn_typ,
            "left_outer",
        )

        join_table2 = join_table.select(
            col("instun_id"),
            concat(lit("196-"), join_table.instun_id).alias("sor_instun_id"),
            col("instun_nm"),
            col("cntry_cd"),
            col("instn_typ"),
            col("cnterpty_typ").alias("cnterpty_typ1"),
            col("instun_ultimate_parent_nm"),
            col("instun_ultimate_parent_id"),
            col("instun_dirct_parnt_id"),
            col("instun_type_id"),
            col("instun_dirct_parnt_nm"),
        )

        return join_table2

    def join_acbs_account_with_acbs_document(
        self, acbs_account, acbs_document
    ) -> DataFrame:
        join_table = acbs_account.join(
            acbs_document,
            acbs_account.acct_num == acbs_document.obligation_number,
            "left_outer",
        )

        return join_table

    def acbs_document_filter_the_snap_date(self, input_df: DataFrame) -> DataFrame:
    # step 5
    snap_dt = self.dates.report_date.isoformat()
    logger.info('Starting ACBS_DOCUMENT - Filter the snap_date')
    input_df.createOrReplaceTempView('ACBS_DOCUMENT')
    sql_transformer = self.get_transformer(SparkSQLTransformer, 'ACBS_DOCUMENT')
    output_df = sql_transformer.transform_data(
        """select *
           from (select *, rank() over(partition by obligation_number order by document_date_entered desc) as rec_rank
                 from ACBS_DOCUMENT
                 where snap_dt in (select MAX(snap_dt) from ACBS_DOCUMENT where snap_dt <= '{snap_dt}')
                 and document_type_class_code = 'GOPF-BA'
                 and document_description in ('GO', 'REV'))
           where rec_rank = 1"""
    )
    logger.info('Completed ACBS_DOCUMENT = Filter the snap_date')

    return output_df

    def select_ultimate_counterparty(self, joined_ds_typ):
        joined_ds_typ.createOrReplaceTempView("joinedDs_typ")
        logger.debug('Starting required fields for ultimate counterparty')
        sql_transformer = self.get_transformer(SparkSQLTransformer, "joinedDs_typ")
        output_df = sql_transformer.transform_data(
            """SELECT distinct instun_id                        as ultimate_instun_id,
                              sor_instun_id                    as ultimate_sor_instun_id,
                              instun_nm                        as ultimate_instun_nm,
                              cntry_cd                         as ultimate_cntry_cd,
                              instn_typ                        as ultimate_instn_typ,
                              cnterpty_typ1                    as ultimate_cnterpty_typ1,
                              instun_ultimate_parent_nm        as ultimate_instun_ultimate_parent_nm,
                              instun_ultimate_parent_id        as ultimate_instun_ultimate_parent_id,
                              instun_dirct_parnt_id            as ultimate_instun_dirct_parnt_id,
                              instun_type_id                   as ultimate_instun_type_id,
                              instun_dirct_parnt_nm            as ultimate_instun_dirct_parnt_nm
               FROM joinedDs_typ
               where instun_id IN (select distinct instun_ultimate_parent_id from joinedDs_typ)"""
        )
        logger.debug('Completed required fields for ultimate counterparty')

        return output_df

    def ultimate_cpty_ds(self, step19, ultimate_cpty_ds):
        logger.debug(
            "Input Files Count for ultimate_cpty_ds:\nInput_df1:%s\nInput_df2:%s",
            step19.count(),
        ultimate_cpty_ds.count()
    )

    join_table = step19.join(
        ultimate_cpty_ds,
        step19.instun_ultimate_parent_id == ultimate_cpty_ds.ultimate_instun_id,
        "left_outer"
    )

    return join_table

    def select_minerva_counterparty(self, joined_ds2_ult_par, minerva_cnterpty_typ):
        logger.debug('Starting required fields for Minerva counterparty')
        logger.debug(
            "Input df1:\nInput_df2:%s",
            joined_ds2_ult_par.count(),
            minerva_cnterpty_typ.count()
        )

        joined_ds2_ult_par.createOrReplaceTempView("joinedDS2_ult_par")
        minerva_cnterpty_typ.createOrReplaceTempView("minerva_cnterpty_typ")

        sql_transformer = self.get_transformer(SparkSQLTransformer, "joinedDS2_ult_par")
        output_df = sql_transformer.transform_data(
            """select jds.instun_id,
                      jds.sor_instun_id,
                      jds.instun_nm,
                      jds.cntry_cd,
                      jds.instn_typ,
                      jds.cnterpty_typ1,
                      mct.cnterpty_typ2,
                      jds.instun_ultimate_parent_nm,
                      jds.instun_ultimate_parent_id,
                      jds.instun_dirct_parnt_id,
                      jds.instun_type_id,
                      jds.instun_dirct_parnt_nm
               from (SELECT instun_id,
                            sor_instun_id,
                            instun_nm,
                            cntry_cd,
                            instn_typ,
                            cnterpty_typ1,
                            ultimate_instun_type_id,
                      instun_ultimate_parent_nm,
                      instun_ultimate_parent_id,
                      instun_dirct_parnt_id,
                      instun_type_id,
                      instun_dirct_parnt_nm
               FROM joinedDS2_ult_par) jds
               LEFT OUTER JOIN (SELECT instn_typ, cnterpty_typ as cnterpty_typ2 from minerva_cnterpty_typ) mct
               on jds.ultimate_instun_type_id = mct.instn_typ"""
        )

        logger.debug('Completed required fields for Minerva counterparty')

        return output_df
